In [1]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span, DocBin

from IPython.display import Markdown, display
from IPython.display import clear_output

from itertools import combinations
import pandas as pd
from datetime import datetime

import requests
from bs4 import BeautifulSoup

import os
import re
import json
import random

c:\users\justin\appdata\local\programs\python\python36\lib\site-packages\cupy\_environment.py:214: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  'CUDA path could not be detected.'
c:\users\justin\appdata\local\programs\python\python36\lib\site-packages\cupy\_environment.py:214: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  'CUDA path could not be detected.'


In [2]:
spacy.prefer_gpu()
nlp = spacy.load("nl_core_news_lg")

In [3]:
nlp_ministries = spacy.load("..\\data\\spacy labeled\\output\\model-last")
df = pd.read_csv('..\\data\\ocred\\files_df.csv', index_col = 0)

This notebook is for rq 2.2.2

There were a lot of itterations for this extractor. These will be first. At the end, the extractor that was used is implemented

In [4]:
def printHilight(string, extraSpace = True):
    if extraSpace:
        print('\x1b[1;31m'+string + ' ' +'\x1b[0m', end='')
    else:
        print('\x1b[1;31m'+string + '\x1b[0m', end='')

def printHilightUnderline(string):
    print('\033[4m\033[96m'+string  +'\033[96m\x1b[0m' + ' ', end='')
    
def inputHandling(message):
    while(True):
        i = input(message)
        if i == 'q':
            return -1
        
        elif i == '':
            return 0
        
        try:
            i = int(i)
            return i
        
        except:
            print("input number, q or nothing")

In [5]:
# calculates f1, precision and recall with the partial method
def calcResultsMinistries(r):
    precision = (r['correct'] + (0.5 * r['partial'])) / (r['correct']+r['incorrect']+r['spurious'])
    recall = (r['correct'] + (0.5 * r['partial'])) / (r['correct']+r['incorrect']+r['missing'])
    f1 = 2 * ((recall * precision)/(recall + precision))
    r['precision'] = precision
    r['recall'] = recall
    r['f1'] = f1 
    print(f'precision = {precision}, recall = {recall}, f1 = {f1}')
    print(r)
    return r

In [6]:
# returns a list of current dutch ministries and their abbriviations
def getMinisteries(getList):
    
    # get html of the wiki page of dutch ministries
    page = requests.get('https://nl.wikipedia.org/wiki/Lijst_van_Nederlandse_ministeries')
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all("td")[-1]
    
    # get all links in the html
    results.find_all('a', href = True)
    wikis = {}
    
    abrr = []
    for item in str(results.find_all('p')[0]).split('\n')[:-1] + str(results.find_all('p')[1]).split('\n')[1:-1]:
        temp = re.findall('(?<=\()(.*?)(?=\))', item)
        if temp == []:
            abrr.append(None)
        elif temp[-1] == 'Nederland':
            abrr.append(None)
            if 'Overzeese Gebiedsdelen' in item:
                abrr.append(None)
        else:
            abrr.append(temp[-1].replace('&amp;', '&'))


    counter = 0
    for ministerie in results.find_all('a')[:12]:
        wikis[ministerie.text] = {'Link': 'https://nl.wikipedia.org' + ministerie['href'], 'Abbriviation' : abrr[counter]}
        counter += 1
    
    if getList:
        minList = list(wikis.keys()) + [wikis[x]['Abbriviation'] for x in wikis.keys()]
        for x in minList:
            temp+=x.replace(',', '').split(' ')

        return [x.lower() for x in temp if x != 'en' and x != ''] + ['ezk']
    else:
        return wikis
    
minList = getMinisteries(True)

In [7]:

def showMatchesOld(text, matches, minList):   
    charIndex = 0
    mIndex = 0

    text = text.replace('\n', ' ')
    text = text.split(' ')
    flag = False


    for word in text:
        startMatch = matches[mIndex][0]
        endMatch = matches[mIndex][1]
        # continuation of match
        if flag:

            # if end of match
            if endMatch <= charIndex + len(word):
                printHilight(word[:endMatch - charIndex], False)
                print(word[endMatch - charIndex:], end=' ')

                flag = False
                mIndex += 1
            # if end of match
            else:
                printHilight(word)

        # match
        elif startMatch == charIndex and endMatch == charIndex + len(word):
            printHilight(word)
            mIndex += 1

        # first part of a match
        elif startMatch == charIndex:
            printHilight(word)
            flag = True

        # partial match
        elif startMatch < charIndex + len(word):

            print(word[:startMatch - charIndex], end='')
            if endMatch <= charIndex + len(word):
                printHilight(word[startMatch - charIndex:endMatch - charIndex], False)
                print(word[endMatch - charIndex:], end=' ')
                mIndex += 1
            else:
                printHilight(word[startMatch - charIndex:endMatch - charIndex])
                flag = True


        # potential uncaught match
        elif word.lower() in minList:
            printHilightUnderline(word)

        # normal word
        else:
            print(word, end=' ')

        charIndex += len(word) + 1



In [8]:
def findMinisteries(minList):
    
    # gets gazetteers for ministries
    ministeries = getMinisteries(False)
    abrr = [ministeries[x]['Abbriviation'].lower() for x in ministeries if ministeries[x]['Abbriviation'] != None]
    mins = [x.lower() for x in list(ministeries.keys())]
    print(mins)
    
    # variants of the extractor
    variants = [('full', True, True), ('broad', True, False), ('no_prefix', False, False)]
    output = {}

    # if results already exist, this will load them
    # otherwise it will create a new results set
    for v in variants:
        if os.path.isfile('..\\data\\results\\min_old_' + v[0] + '.json'):
            with open('..\\data\\results\\min_old_' + v[0] + '.json') as f:
                output[v[0]] = json.load(f)
            print('Loaded previous output for ', v[0])
        else:
            print('new output for ' + v[0])
            output[v[0]] = {
                'abbr':v[1],
                'lookForMin':v[2],
                'correct':0,
                'partial':0,
                'missing':0,
                'spurious':0,
            }
    

    minList = set(getMinisteries(True))

    # one loop is one document
    while(True):
        try:

            # sample a random document from the df
            sample = df.sample(1)
            text = sample.text.values[0]

            # remove exces newlines and spaces
            text = re.sub('\n+', '\n', text)
            text = re.sub(' +', ' ', text)

            for m in minList:
                if m in text.split(' '):
                    break
            else:
                continue
            
            # loop for all extractors
            for i in output:

                # set result to current extractor
                results = output[i]

                # print some info
                print('version', i)
                print(sum(list(results.values())[2:6] ))
                print(sample.name.values[0], sample.page.values[0])

                # settings for extractor
                if results['abbr']:
                    allMinisteries = mins + abrr
                else:
                    allMinisteries = mins

                matches = []

                # look for all ministries
                for ministerie in allMinisteries:
                    
                    # more settings for extractor
                    if results['lookForMin']:        
                        matches += [(x.start(), x.end()) for x in re.finditer('ministerie van ' + ministerie.lower(),text.lower())]       
                    else:
                        matches += [(x.start(), x.end()) for x in re.finditer(ministerie.lower(),text.lower())]    

                # get number of matches and pring
                print('n matches: ', len(matches), '\n\n')

                # sort matches and show document
                matches = sorted(matches, key=lambda tup: tup[0])
                matches.append((len(text), len(text) + 1))
                showMatchesOld(text, matches, minList)

                # inputs for performance
                for case in list(results.keys())[2:6]:
                    result = inputHandling(case)

                    # if input function returns -1, the program stops
                    if result == -1:
                        output[i] = results
                        return output
                    else:
                        results[case] += result

                # clear output for next loop
                clear_output()
        except:
            pass


In [9]:
results = findMinisteries(minList)

for variant in results:
    output = calcResultsMinistries(results[variant])
    with open('..\\data\\results\\min_old_' + variant + '.json', 'w') as f:
        json.dump(output, f)
    print('______________________________')


version no_prefix
108
07e2b274045cb5b4f54371a3c905cae9_wobverzoek-mccb-catshuis 2
n matches:  0 


document 02 Sitrap SZW COVID-19, nummer 1   Van: Departementaal Crisiscentrum (DCC) Aan: leden Bewindspersonenstaf, leden Bestuursraad, directeuren SZW Datum: 1 maart, 17.00 Departementaal vertrouwelijk   Dit is een eerste sitrap, op basis van de thans bij het DCC bekende informatie. Het beeld zal de komende tijd telkens worden aangevuld met actuele informatie. Zo lang relevant, wordt voor elke bewindspersonenstaf en BR een actuele sitrap rondgestuurd. Aanvullingen of wijzigingen kunnen worden aangeleverd bij dcc@minszw.nl of 1022 @minszw.nl   Crisisstructuur SZW:   De NCTV en VWS zijn leidend in de interdepartementale crisisstructuur. DCC SZW coördineert de aanpak binnen SZW, is de verbindingsschakel van de interdepartementale crisisstructuur naar SZW en zorgt voor het informeren van de politieke en ambtelijke leiding SZW. De directeur BO is hoofd DCC. De directies blijven verantwoordeli

In [9]:
minList

['wvc',
 'algemene',
 'zaken',
 'binnenlandse',
 'zaken',
 'koninkrijksrelaties',
 'buitenlandse',
 'zaken',
 'defensie',
 'economische',
 'zaken',
 'klimaat',
 'financiën',
 'infrastructuur',
 'waterstaat',
 'justitie',
 'veiligheid',
 'landbouw',
 'natuur',
 'voedselkwaliteit',
 'onderwijs',
 'cultuur',
 'wetenschap',
 'sociale',
 'zaken',
 'werkgelegenheid',
 'volksgezondheid',
 'welzijn',
 'sport',
 'az',
 'bzk',
 'bz',
 'def',
 'ez',
 'fin',
 'i&w',
 'j&v',
 'lnv',
 'ocw',
 'szw',
 'vws',
 'ezk']

## Conclusion

It is easy to find correct matches for ministeries when using gazetteers. The precision of this test is 0.882. Most all of the matches that were found are actually ministeries. However the recall of the matcher is very bad at 0.377. The matcher doesnt even find half of all the ministeries that were in the text. The reason for this is clear. It has to do with summations as abbriviations. 

Within the texts, a lot of summations of ministeries are used. For example: "de ministeries van VWSS, JenV, en BZK". In this case none of these will abbreviations of ministeries will be matched. The matcher looks for "ministerie van {name of ministerie}". VWSS will not be matched because ministeries is plural in stead of singular and JenV and BZK will not be matched because these are not preceded by "ministerie van ". A solution for this could be to not look for the preceding "ministerie van ", however, if this is done, all of those abbriviations will be matched outside of context. The abbriviation of the ministery of defence for example, is "def" so the matcher will find all occurences of the three letters "def". With this modification the recall will actually increase to a whopping 0.458 but the precision will decrease to a meager 0.122. So this is not a solution. 

I also tested if the it would help to at least find the VWSS in the example by also looking at the plural of ministerie but that decreased the precision more than it increased the recall. 

The last solution is the "best". This completely ignores the abbriviations and just looks for the names if the ministeries without the preceding "ministerie van ". This actually increases the recall to 0.435 but it also decreases the precision to 0.732. This is because it will also find random mentions of the words "financien" and "defensie" and others. I don't think that that is worth it. In this case I would rather have more confidance in what I what I find to be correct than finding everything.

Now for the best extractor

## Labeling new data

In [11]:
# this function allows a user to label new data
def showTokens(text, allMinisteries):

    try:
        text = re.sub(' +', ' ', text)
        text = text.replace('\n', ' ')
    except:
        pass
    doc = nlp(str(text))
    # get list of entities as strings
    listTokens = [str(ent.text) for ent in doc.ents]

    # control variables
    nToken = 0
    tempTokens = []
    lenTemp = 0

    # loop over tokens
    for i in range(len(doc)):

        # get string representation of token and add to a temporary list with token index
        t = str(doc[i])
        tempTokens.append((t, nToken))

        # increase n tokens and length of all strings in tempTokens
        nToken += 1
        lenTemp += (len(t) + 1)

        # once lenght of all strings in tempTokens is 100 or larger, start printing the line
        if lenTemp >= 100 or i >= len(doc) - 1:

            # print the index for every token
            for word in tempTokens:

                # if index number is more chars than the token, dont print the index number
                if len(word[0]) < len(str(word[1])):
                    print(' ' * (len(word[0]) + 1), end='')

                # else just print index number plus a number of spaces
                else:
                    print(word[1], ' ' * (len(word[0]) - len(str(word[1]))), end='')
            print('')

            # print all tokens
            for word in tempTokens:

                # highlight tokens in red if named entity
                if word[0].lower() in allMinisteries + ['ministerie', 'ministeries']:
                    printHilight(word[0])
                else:
                    print(word[0] + ' ', end='')

            # add some space between lines
            print('\n\n')

            # reset control variables
            tempTokens = []
            lenTemp = 0
    
    # inputs:
    # no input: next page
    # q: quit 
    # remove: remove last label
    # number1 number2: adds a label from span number1 to number2
    # everything else is invalid, you will be prompted again
    
    
    newEnts = []
    while True:
        x = input()
        if x == '':
            break
        elif x == 'q':
            raise Exception("Stopped the program")
        elif x == 'remove':
            del newEnts[-1]
            print('Removed')
            continue
        
        try:
            x = x.split()
            newEnts.append(Span(doc, int(x[0]), int(x[1]), label='MINISTERIE'))
            print(doc[int(x[0]):int(x[1])])
        except:
            print('Two numbers seperated by a space')
            continue
            
        
    try:
        doc.ents = newEnts
        clear_output()
    except:
        pass
        
    return doc

In [12]:
def showText(df, n):
    
    ministeries = getMinisteries()
    abrr = [ministeries[x]['Abbriviation'] for x in ministeries if ministeries[x]['Abbriviation'] != None]
    allMinisteries = list(ministeries.keys()) + abrr
    allMinisteries = [x.lower() for x in allMinisteries]
    
    samples = df.sample(n)
    docs = list(samples.text.apply(lambda x: showTokens(x, allMinisteries)))
    return docs

In [ ]:
docs = []

for i in range(30):
    docs += showText(df, 10)
    print(f'done {i + 1}0')

In [ ]:
random.shuffle(docs)
train_docs = docs[:len(docs) // 2]
dev_docs = docs[len(docs) // 2:]

# Create and save a collection of training docs
train_docbin = DocBin(docs=train_docs)
train_docbin.to_disk("..\\data\\spacy labeled\\train2.spacy")
# Create and save a collection of evaluation docs
dev_docbin = DocBin(docs=dev_docs)
dev_docbin.to_disk("..\\data\\spacy labeled\\dev2.spacy")

In [13]:
# nlp1 = spacy.load("..\\data\\spacy labeled\\output\\model-last")

## Evaluating new model

In [1]:
def showMatchesMinistries(doc, minList):   
    indexOfMatches = []
    for ent in doc.ents:
        for i in range(int(ent.start), int(ent.end)):
            indexOfMatches.append(i)

    indexOfMatches = set(indexOfMatches)

    for token in doc:
        flag = False
        for mini in minList:
            if token.text.lower() == mini:
                flag = True
                break
        
        if token.i in indexOfMatches:
            printHilight(str(token.text))
            
        elif flag:
            printHilightUnderline(str(token.text))
            
        else:
            print(token, end=' ')
    
    return
    

In [15]:
def evaluateMinistries(nlp):
    # cor = exact match, inc = match is wrong (wrong bounds or label), mis = missing match, spu = found something that
    # isnt a mactch, 
    if os.path.isfile('..\\data\\results\\ministries_results,json'):
        with open('..\\data\\results\\ministries_results,json') as f:
            results = json.load(f)
        
        pass
    else:
        results = {
            'correct':0,
            'incorrect':0,
            'partial':0,
            'missing':0,
            'spurious':0,
            'abbr' :0,
            'ocr':0,
            'whitespace':0,
            'minister':0,
            'other':0
        }
    
    minList = set(getMinisteries(True))
    while(True):
        try:
            sample = df.sample(1)
            text = sample.text.values[0]
            text = re.sub('\n+', '\n', text)
            text = re.sub(' +', ' ', text)
            for m in minList:
                if m in text.split(' '):
                    break
            else:
                continue
                
            print(sum(list(results.values())[:4] ))
            print(sample.name.values[0], sample.page.values[0], '\n\n')
            doc = nlp(text)
            
            showMatchesMinistries(doc, minList)

            for case in list(results.keys())[:10]:
                result = inputHandling(case)
                if result == -1:
                    return results
                else:
                    results[case] += result

            clear_output()
        except:
            continue
            
        

In [16]:
results = evaluateMinistries(nlp_ministries)

results = calcResultsMinistries(results)

with open('..\\data\\results\\ministries_results,json', 'w') as f:
    json.dump(results, f)

300
306d4b337a5d407eb00c8845884c82fd_rivm-maart-2020-documenten 2674 


( instabiliteit , onveiligheid , economische verstrengeling met elkaar , toegang tot markten in de toekomst ) moeten we ons ook 
 bezig houden met een internationale aanpak . We moeten dit nu internationaal regelen met het IMF , Wereldbank en de EU , 
 dan zullen we nog heel lang de gevolgen hiervan plukken . 
 * “ Binnen het budget dat ik heb , kijken we naar andere middelen . We kijken ook juist naar hoe het op een Marshall-achtige 
 fund aangepakt kan worden door het IMF . ” 
 « Social distancing is een basisconcept dat in vluchtelingenkampen en favela's wegvalt . Structurele armoede en slecht bestuur 
 is een giftige mengeling . 
 s De cyclus van een aanpak van corona , morgen misschien een ander virus , is ook afhankelijk van de aanpak in de zwakste 
 schakel in de keten . Wervenheid van reizen , werken , denken en mensen , moet je ook aan de zwakste schakel werken . Groot 
 punt van zorg . Ook delen van lesse

## Conclusion
Missing things in situations like this: RVO/LNV. LNV should be have been caught here. This might be caused by the fact that spacy works with tokens/words in stead of individual letters. Places with a lot of extra newlines or spaces within the name of a ministry will also trip up the model. No context also doesnt help

when training I specificly excluded the minister variants of the ministries. Therefore these aren't counted as missing if the model doesn't find them but will be counted as spurious if the the model does find them.

The data the model was trained on means that it easily recoginizes ministries that have a lot to do with covid like volksgezondheid, but less so for others like landbouw. EZK (economics and climate) specifically doesn't want to be captured

The type of text doesnt lend itself very well to NER. A lot of times the abbriviations of the ministries are listed without any contex to what it is. NER uses information about the surrounding words to predict what the word is. If there are no surrounding words, no context, predictions are going to be difficult

ministerie SZW en EZK was counted as two partially correct predictions.